In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('../input/new-york-ecommerce-customers/Ecommerce Customers')
print('setup complete')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.info()
#500 rows with 8 columns

Are there any null values?
No

In [ ]:
df.isnull().sum().sum()


# EXPLORATORY DATA ANALYSIS

Create some plots to see the relationships between columns

In [ ]:
sns.pairplot(df)

Looks like we have some correlation between y-axis Yearly Amount Spent and x-axis Length of Membership

Let's make a heatmap of these correlations

In [ ]:
sns.heatmap(df.corr(),annot=True,cmap="YlGnBu")

What about low correlation? Compare Time on Website and Yearly Amount Spent. Is there a correlation?

In [ ]:
sns.jointplot(data=df, x='Time on Website', y='Yearly Amount Spent')

A little more correlation between Time on App and Yearly Amount Spent

In [ ]:
sns.jointplot(data=df, x='Time on App', y='Yearly Amount Spent')

In [ ]:
sns.jointplot(data=df, x='Time on App', y='Length of Membership', kind='hex')

In [ ]:
sns.lmplot(data=df,x='Length of Membership', y='Yearly Amount Spent')

Good linear fit with little error within the error bar.  Longer you stay a member the higher the yearly amount spenditure.

# SPLITTING THE DATA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X =df[['Avg. Session Length','Time on App','Time on Website','Length of Membership']]
y =df['Yearly Amount Spent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)



# TRAINING THE MODEL

In [ ]:
lm_model=LinearRegression() #create lr object (model)
lm_model.fit(X_train, y_train) #train the model (fit it)

In [ ]:
lm_model.coef_

# PREDICTING THE MODEL

In [ ]:
lm_predictions=lm_model.predict(X_test)


Scatterplot of real test values (y) vs. the predicted test values

In [ ]:
plt.scatter(y_test, lm_predictions)
plt.xlabel('Y Test(True Values)')
plt.ylabel('Predicted Values')

# EVALUATING THE MODEL
Mean Absolute Error (MAE) is the mean of the absolute value of the errors. Measures the difference between 2 constant variables. Is the easiest to understand, because it's the average error. Here actual and predicted values of y.

Mean Squared Error (MSE) is the mean of the squared errors:MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.

Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors:RMSE** is even more popular than MSE, because RMSE is interpretable in the y units.

These are loss functions because you want to minimize them to create the best model

In [ ]:
from sklearn import metrics

print('MSE', metrics.mean_squared_error(y_test, lm_predictions))
print('MAE', metrics.mean_absolute_error(y_test, lm_predictions))
print('RMSE', np.sqrt(metrics.mean_squared_error(y_test, lm_predictions)))

In [ ]:
metrics.explained_variance_score(y_test, lm_predictions)

# EVALUATING RESIDUALS
Explore the residuals to make sure our model was correct for our data

Histogram of the residuals to make sure it is normally distributed

In [ ]:
sns.distplot(y_test-lm_predictions)

In [ ]:
cdf=pd.DataFrame(lm_model.coef_, X_train.columns, columns=['Coeff'])

cdf

If all other features are fixed then with an increase of 1 unit the coefficient increases by set amount.
Example: With Avg. Session Length increase of 1 unit, the monetary increase will be $25.981550 (with all other features fixed)

Length of Membership has largest coefficient

Time of Website is lowest

Company should focus on either increasing time on website or continue to gain monetary income over time on app
